In [1]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import kaggle_metric
from utils import sigmoid

In [2]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
sample_submission = pd.read_csv('../../input/kaggle/sample_submission.csv')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [9]:
sub = pd.read_csv('submission.csv', index_col='recording_id')
display(sub)
display(sub.describe())

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,,,,
000316da7,0.008587,0.000421,0.001058,0.064189,0.000710,0.002391,0.000271,0.001751,0.001351,0.013146,0.001420,0.000559,0.023772,0.003312,0.001042,0.008835,0.005026,0.001649,0.038595,0.001490,0.005878,0.002940,0.005518,0.009847
003bc2cb2,0.002640,0.002776,0.000629,0.011313,0.000744,0.003099,0.000688,0.001559,0.002714,0.002162,0.002052,0.003179,0.000815,0.004398,0.009659,0.009074,0.559759,0.001716,0.006805,0.000943,0.002371,0.005375,0.002583,0.003941
0061c037e,0.019482,0.005551,0.001663,0.002905,0.000772,0.007853,0.000972,0.019437,0.002781,0.004910,0.505219,0.005435,0.002450,0.002408,0.000386,0.005718,0.023660,0.108431,0.002077,0.011787,0.019077,0.004266,0.001314,0.004584
010eb14d3,0.263609,0.002066,0.002997,0.050433,0.003897,0.015037,0.002095,0.021849,0.513702,0.004830,0.001706,0.002320,0.040574,0.048702,0.006412,0.001653,0.042593,0.000795,0.116849,0.002270,0.021308,0.022187,0.000580,0.011629
011318064,0.002469,0.002168,0.002270,0.035509,0.000909,0.002123,0.002763,0.000880,0.008249,0.002858,0.003241,0.003264,0.002655,0.002981,0.312324,0.051277,0.005945,0.003241,0.082074,0.001423,0.005716,0.015804,0.001549,0.002655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,0.001778,0.002258,0.005117,0.055032,0.004591,0.410201,0.000565,0.010169,0.008106,0.001583,0.007666,0.000398,0.005556,0.007974,0.000767,0.439122,0.004671,0.004636,0.014960,0.003638,0.002491,0.003162,0.000649,0.728038
ff973e852,0.002444,0.001222,0.000598,0.007844,0.000367,0.003635,0.001180,0.095161,0.000956,0.031531,0.003972,0.002542,0.001936,0.003091,0.007618,0.033080,0.004447,0.043540,0.003138,0.003471,0.004100,0.002272,0.004724,0.004074
ffa5cf6d6,0.003748,0.063266,0.001449,0.038337,0.000319,0.002223,0.000608,0.000701,0.001221,0.014593,0.001068,0.000577,0.001637,0.004860,0.000639,0.243258,0.068357,0.072517,0.001822,0.003633,0.002002,0.004172,0.004200,0.006535


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
count,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000
mean,0.026391,0.025144,0.024059,0.044844,0.008971,0.029602,0.004187,0.051641,0.023651,0.017795,0.020312,0.027184,0.047458,0.024018,0.032997,0.032904,0.046383,0.016008,0.046551,0.007503,0.011731,0.016782,0.020061,0.041484
std,0.078321,0.072140,0.116955,0.031077,0.061859,0.094580,0.033674,0.100290,0.101506,0.069198,0.101150,0.089875,0.102411,0.085653,0.101335,0.084366,0.143305,0.082360,0.079991,0.055329,0.058348,0.065465,0.099135,0.142211
min,0.000590,0.000149,0.000209,0.001249,0.000107,0.000461,0.000130,0.000402,0.000202,0.000475,0.000303,0.000165,0.000357,0.001042,0.000137,0.000738,0.001619,0.000359,0.000308,0.000345,0.000652,0.001082,0.000419,0.001108
25%,0.002009,0.000852,0.000918,0.023320,0.000610,0.002432,0.000582,0.001897,0.000972,0.002544,0.001463,0.000714,0.001535,0.003682,0.000843,0.002465,0.006337,0.001731,0.002489,0.001301,0.002318,0.003894,0.001525,0.003325
50%,0.003471,0.002085,0.001633,0.040717,0.001067,0.004559,0.001040,0.005368,0.002036,0.004233,0.002739,0.001381,0.002970,0.006237,0.001885,0.004264,0.010007,0.002852,0.009848,0.002161,0.003876,0.006592,0.002622,0.005274
75%,0.007328,0.008948,0.003542,0.060084,0.002393,0.009931,0.002090,0.034685,0.004979,0.008180,0.005616,0.004128,0.012833,0.015084,0.005194,0.011303,0.018138,0.005467,0.054994,0.003580,0.006858,0.011757,0.004640,0.010391
max,0.798606,0.684047,0.917794,0.225931,0.941641,0.778040,0.827265,0.535654,0.766380,0.855036,0.945692,0.777921,0.509381,0.972838,0.687830,0.636834,0.936304,0.973104,0.749802,0.975972,0.958797,0.896366,0.949327,0.957752


In [10]:
sub.mean()

s0     0.026391
s1     0.025144
s2     0.024059
s3     0.044844
s4     0.008971
s5     0.029602
s6     0.004187
s7     0.051641
s8     0.023651
s9     0.017795
s10    0.020312
s11    0.027184
s12    0.047458
s13    0.024018
s14    0.032997
s15    0.032904
s16    0.046383
s17    0.016008
s18    0.046551
s19    0.007503
s20    0.011731
s21    0.016782
s22    0.020061
s23    0.041484
dtype: float64

In [11]:
sub.mean().mean()

0.026985809843440733